### Break repeating-key XOR

#### It is officially on, now.

This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to **qualify** you. If you can do this one, you're probably just fine up to Set 6.

[There's a file here](challenge-6-file.txt). It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.

2. Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:

    ```
    this is a test
    ``` 

    and

    ```
    wokka wokka!!!
    ```

    is **37**. Make sure your code agrees before you proceed.

3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
7. Solve each block as if it was single-character XOR. You already have code to do this.
8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.


This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.

In [115]:
import base64
import itertools

def make_blocks_list(buffer, block_size):
    return [buffer[n:n+block_size] if n+1 < len(buffer) else buffer[n:] for n in range(0, len(buffer), block_size)]

def str_to_bin(buffer):
    bin_list = [format(b, 'b').zfill(8) for b in buffer]
    return ''.join(bin_list).encode()

def haming_distance(buffer_1, buffer_2):
    bin_1 = str_to_bin(buffer_1)
    bin_2 = str_to_bin(buffer_2)
    return len(list(filter(lambda x: x[0]^x[1], zip(bin_1, bin_2))))

assert(haming_distance(b"this is a test", b"wokka wokka!!!") == 37)

def get_average_distance(buffer, keysize):
    blocks = [buffer[n:n+keysize] for n in range(0, len(buffer) - len(buffer) // keysize, keysize)]
    results = [haming_distance(blocks[n], blocks[n+1]) / keysize for n in range(0, len(blocks) - len(blocks) // 2, 2)]
    return sum(results) / len(results)

def find_key_size(buffer):
    results = []
    for keysize in range(2, 41):
        results.append(
            (keysize, get_average_distance(buffer, keysize))
        )
    results.sort(key=lambda x: x[1])
    return results[0][0]

def transpose_blocks(buffer_blocks):
    transposed_blocks = [[]] * len(buffer_blocks[0])
    for i in range(len(buffer_blocks[0])):
        for block in buffer_blocks:
            if not transposed_blocks[i]:
                transposed_blocks[i] = []
            if i < len(block):
                transposed_blocks[i].append(block[i])

    return transposed_blocks


def decrypt(buffer, passphrase):
    if type(passphrase) != list:
        passphrase = [passphrase]
    character = itertools.cycle(passphrase)
    return bytes([byte ^ next(character) for byte in buffer])

def get_english_score(input_bytes):
    """Compares each input byte to a character frequency 
    chart and returns the score of a message based on the
    relative frequency the characters occur in the English
    language
    """
    character_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    return sum([character_frequencies.get(chr(byte), 0) for byte in input_bytes.lower()])

def decrypt_single_character(buffer):
    scores = []
    for key in range(32, 127):
        score = get_english_score(decrypt(buffer, key))
        scores.append(
            (key, score)
        )

    scores.sort(key=lambda x: x[1], reverse=True)
    key = scores[0][0]
    return key

def find_key(encrypted_buffer):
    keysize = find_key_size(data)
    buffer_blocks = make_blocks_list(data, keysize)
    transposed_blocks = transpose_blocks(buffer_blocks)

    print("Keysize: {}\n# of blocks: {}, # of transposed blocks: {}".format(keysize, len(buffer_blocks), len(transposed_blocks)))

    passphrase = []
    for block in transposed_blocks:
        ch = decrypt_single_character(block)
        passphrase.append(ch)
    print("Passphrase: {}\n".format(bytes(passphrase)))
    return passphrase


with open("challenge-6-file.txt", "r") as infile:
    b64_encoded = infile.read().replace("\n", "")
data = base64.b64decode(b64_encoded)

passphrase = find_key(data)

print(decrypt(data, passphrase).decode())


Keysize: 29
# of blocks: 100, # of transposed blocks: 29
Passphrase: b'Terminator X: Bring the noise'

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalaf